In [1]:
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
import numpy as np
import cv2
import os

In [10]:
prototxtPath=os.path.sep.join([r'C:\Users\kumar\mask_detector\Facedetector','architecture.txt'])
weightsPath=os.path.sep.join([r'C:\Users\kumar\mask_detector\Facedetector','weights.caffemodel'])

In [11]:
prototxtPath

'C:\\Users\\kumar\\mask_detector\\Facedetector\\architecture.txt'

In [12]:
weightsPath

'C:\\Users\\kumar\\mask_detector\\Facedetector\\weights.caffemodel'

In [13]:
net=cv2.dnn.readNet(prototxtPath,weightsPath)

In [14]:
model=load_model(r'C:\Users\kumar\mask_detector\Transfer_learning')

In [16]:
image=cv2.imread(r'C:\Users\kumar\mask_detector\data\with_mask\with_mask_1.jpg')

In [17]:
image

array([[[38, 66, 60],
        [35, 63, 57],
        [32, 59, 50],
        ...,
        [11, 37, 31],
        [18, 41, 36],
        [13, 29, 22]],

       [[35, 63, 57],
        [32, 60, 54],
        [31, 58, 49],
        ...,
        [15, 38, 33],
        [21, 44, 36],
        [15, 31, 24]],

       [[32, 58, 52],
        [31, 57, 51],
        [31, 55, 47],
        ...,
        [17, 40, 32],
        [24, 45, 37],
        [20, 34, 28]],

       ...,

       [[30, 71, 56],
        [31, 72, 57],
        [35, 71, 57],
        ...,
        [12, 12, 18],
        [12, 12, 18],
        [12, 12, 18]],

       [[27, 69, 52],
        [28, 70, 53],
        [32, 68, 54],
        ...,
        [13, 12, 21],
        [13, 12, 21],
        [13, 12, 21]],

       [[26, 68, 51],
        [27, 69, 52],
        [31, 67, 53],
        ...,
        [13, 12, 21],
        [13, 12, 21],
        [13, 12, 21]]], dtype=uint8)

In [39]:
(h,w)=image.shape[:2]

In [40]:
(h,w)

(350, 525)

In [33]:
blob = cv2.dnn.blobFromImage(image, 1.0, (300, 300))

In [34]:
blob

array([[[[37., 31., 29., ..., 30., 11., 15.],
         [33., 31., 30., ..., 22., 14., 18.],
         [30., 30., 30., ..., 10., 17., 24.],
         ...,
         [31., 36., 35., ..., 11., 11., 11.],
         [28., 33., 33., ..., 13., 13., 13.],
         [26., 31., 29., ..., 13., 13., 13.]],

        [[64., 58., 53., ..., 54., 37., 33.],
         [61., 57., 54., ..., 44., 38., 36.],
         [56., 54., 54., ..., 30., 39., 40.],
         ...,
         [72., 72., 68., ..., 12., 11., 11.],
         [70., 68., 66., ..., 13., 12., 12.],
         [68., 67., 62., ..., 13., 12., 12.]],

        [[58., 49., 43., ..., 49., 31., 27.],
         [55., 48., 44., ..., 38., 32., 29.],
         [50., 46., 45., ..., 22., 32., 33.],
         ...,
         [57., 58., 57., ..., 16., 17., 17.],
         [53., 54., 52., ..., 19., 20., 20.],
         [51., 53., 49., ..., 19., 21., 21.]]]], dtype=float32)

In [35]:
blob.shape

(1, 3, 300, 300)

In [36]:
net.setInput(blob)
detections=net.forward()


In [37]:
detections

array([[[[0.        , 1.        , 0.89960504, ..., 0.16264474,
          0.6816136 , 0.56650895],
         [0.        , 1.        , 0.1274717 , ..., 4.001155  ,
          4.83672   , 4.9880867 ],
         [0.        , 1.        , 0.12440115, ..., 4.001961  ,
          0.8342402 , 4.987373  ],
         ...,
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ],
         [0.        , 0.        , 0.        , ..., 0.        ,
          0.        , 0.        ]]]], dtype=float32)

In [38]:
detections.shape

(1, 1, 200, 7)

In [57]:

#loop over the detections
for i in range(0,detections.shape[2]):
    confidence=detections[0,0,i,2]
    
    
    if confidence>0.5:
        #we need the X,Y coordinates
        box=detections[0,0,i,3:7]*np.array([w,h,w,h])
        (startX,startY,endX,endY)=box.astype('int')
        
        #ensure the bounding boxes fall within the dimensions of the frame
        (startX,startY)=(max(0,startX),max(0,startY))
        (endX,endY)=(min(w-1,endX), min(h-1,endY))
        
        
        #extract the face ROI, convert it from BGR to RGB channel, resize it to 112,112 and preprocess it
        face=image[startY:endY, startX:endX]
        face=cv2.cvtColor(face,cv2.COLOR_BGR2RGB)
        face=cv2.resize(face,(112,112))
        face=img_to_array(face)
        face=preprocess_input(face)
        face=np.expand_dims(face,axis=0)
        output=model.predict(face)
        
        mask=output[0][0]
        withoutMask=output[0][1]
        
        #determine the class label and color we will use to draw the bounding box and text
        label='Mask' if mask>withoutMask else 'No Mask'
        color=(0,255,0) if label=='Mask' else (0,0,255)
        
        #include the probability in the label
        label="{}: {:.2f}%".format(label,max(mask,withoutMask)*100)
        
        #display the label and bounding boxes
        cv2.putText(image,label,(startX,startY-10),cv2.FONT_HERSHEY_SIMPLEX,0.45,color,2)
        cv2.rectangle(image,(startX,startY),(endX,endY),color,2)
        
        
        
cv2.imshow("OutPut",image)
cv2.waitKey(0)
cv2.destroyAllWindows()